## Perceptron wielowarstwowy
#### do predykcji *czego właściwie?*
Maciej Komosa, Informatyka i Ekonometria, 3 rok, stacjonarnie, WZ AGH

1. Importowanie bibliotek oraz kodu funkcji aktywacji

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
%run Projekt_SSN.ipynb

#### Neuron, z metodami propagacji naprzód i wstecz
Przy propagacji naprzód zapisuje w obiekcie wielkość wejscia oraz wyjscia, zwraca wyjscie
Przy propagacji wstecz zapisuje deltę, oblicza gradient dla wag oraz dla biasu oraz aktualizuje wagi i bias zgodnie z obliczonymi wartościami, zwraca sumę produktów szeregu wag oraz delty

In [ ]:
class neuron:
    def __init__(self, wagi, bias, funkcja_aktywacji):
        self.wagi = wagi
        self.bias = bias
        self.funkcja_aktywacji = funkcja_aktywacji
        self.wejscie = []
        self.wyjscie = 0
        self.delta = 0

    def naprzod(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = self.funkcja_aktywacji(np.sum(np.dot(self.wagi, self.wejscie)) + self.bias)
        return self.wyjscie

    def wstecz(self, blad, kolejne_wagi = 1, wspolczynnik_uczenia = 0.1):
        self.delta = np.sum(np.dot(kolejne_wagi, blad)) * self.funkcja_aktywacji(self.wyjscie, pochodna=True)
        print('kolejne wagi= ', kolejne_wagi, 'blad= ', blad, 'wejscie= ', self.wejscie, 'wyjscie= ', self.wyjscie, 'dot= ', np.sum(np.dot(kolejne_wagi, blad)), 'delta= ', self.delta)
        gradient_wagi = np.dot(self.delta, self.wejscie)
        gradient_bias = self.delta
        print('przed wagi= ', self.wagi, 'bias= ', self.bias)
        self.wagi -= wspolczynnik_uczenia * gradient_wagi
        self.bias -= wspolczynnik_uczenia * gradient_bias
        print('po wagi= ', self.wagi, 'bias= ', self.bias)
        #print('wagi*delta= ', np.sum(self.wagi * self.delta))
        return np.sum(self.wagi * self.delta)

#### Perceptron, z metodami propagacji naprzód oraz wstecz
Przy propagacji naprzód przepuszcza dane wejściowe przez całą sieć, zwraca wyjście ostatniej warstwy (ostatnia warstwa jest jednym neuronem, wyjście jest jedną liczbą)
Przy propagacji wstecz oblicza błąd predykcji oraz przepuszcza go przez całą sieć, nie zwraca żadnej wartości

In [ ]:
class wielowarstwowy_perceptron:
    def __init__(self, n_warstw, n_neuronow, n_wejsc, n_wyjsc, funkcja_aktywacji):
        self.warstwy = []
        warstwa = []
        # warstwa wejscia
        for i in range (n_wejsc):
            warstwa.append(neuron(np.random.rand(), np.random.rand(), funkcja_aktywacji))
        self.warstwy.append(warstwa)
        # warstwy ukryte
        for i in range(n_warstw):
            warstwa = []
            for _ in range(n_neuronow):
                if i == 0:
                    warstwa.append(neuron(np.random.rand(n_wejsc), np.random.rand(), funkcja_aktywacji))
                else:
                    warstwa.append(neuron(np.random.rand(n_neuronow), np.random.rand(), funkcja_aktywacji))
            self.warstwy.append(warstwa)
        warstwa = []
        # warstwa wyjscia
        for i in range(n_wyjsc):
            warstwa.append(neuron(np.random.rand(n_neuronow), np.random.rand(), funkcja_aktywacji))
        self.warstwy.append(warstwa)
        self.funkcja_aktywacji = funkcja_aktywacji

    def naprzod(self, wejscie):
        wyjscie_warstwy = []
        # pierwsza warstwa
        for i, neuron in enumerate(self.warstwy[0]):
            wyjscie_warstwy.append(neuron.naprzod(wejscie[i]))
        # warstwy ukryte
        for i, warstwa in enumerate(self.warstwy[1:]):
            nowe_wyjscie_warstwy = []
            for neuron in warstwa:
                nowe_wyjscie_warstwy.append(neuron.naprzod(wyjscie_warstwy))
            wyjscie_warstwy = nowe_wyjscie_warstwy
            
        return self.warstwy[-1][0].wyjscie

    def wstecz(self, wejscie, wspolczynnik_uczenia):
        print('NOWY PRZEBIEG')
        print('wejscie= ', wejscie, 'wyjscie= ', self.warstwy[-1][0].wyjscie)
        blad = np.sum(np.abs(self.warstwy[-1][0].wyjscie - wejscie) * self.warstwy[-1][0].funkcja_aktywacji(self.warstwy[-1][0].wyjscie, pochodna=True))
        # ostatnia warstwa posiada jeden neuron
        blad = self.warstwy[-1][0].wstecz(blad, wspolczynnik_uczenia=wspolczynnik_uczenia)
        wagi = self.warstwy[-1][0].wagi
        # propagacja błędu wstecz
        for i, warstwa in enumerate(reversed(self.warstwy[1:-1])):
            print('warstwa ', i + 1)
            nowy_blad = np.zeros(len(warstwa))
            nowe_wagi = np.zeros((len(warstwa), len(warstwa[0].wagi)))
            for j, neuron in enumerate(warstwa):
                if i == 0:
                    nowy_blad[j] = neuron.wstecz(blad, kolejne_wagi=wagi[j], wspolczynnik_uczenia=wspolczynnik_uczenia)
                else:
                    nowy_blad[j] = neuron.wstecz(blad, kolejne_wagi=wagi[:, j], wspolczynnik_uczenia=wspolczynnik_uczenia)
                nowe_wagi[j] = neuron.wagi
            blad = nowy_blad
            wagi = nowe_wagi

#### Funkcje trenowania i testowania
Funkcja trenująca przepuszcza przez model dane trenujące wprzód oraz cel wstecz (algorytm trenowania implementowany jest w sieci oraz neuronach w metodzie propagacji wstecznej)
Funkcja testująca przepuszcza przez model dane testowe wprzód i zestawia wartości oczekiwane z rzeczywistymi

In [ ]:
def trenuj(model, X, y, n_okresow, wspolczynnik_uczenia, batch=1):
    for _ in tqdm(range(n_okresow)):
        for i in range(1, len(X), batch):
            model.naprzod(X[i])
            model.wstecz(y[i], wspolczynnik_uczenia=wspolczynnik_uczenia)
            

def test(model, X, y):
    for xi, yi in zip(X, y):
        wyjscie = model.naprzod(xi)
        print("Input: {} Output: {} Expected: {}".format(xi, wyjscie, yi))

#### Obliczenia, próba optymalizacji wartości hiperparametrów

Przygotowanie danych

In [ ]:
from sklearn.preprocessing import MinMaxScaler
dane = pd.read_csv('housing_price_dataset.csv')

enkoder_etykiet = LabelEncoder()
for kolumna in dane.select_dtypes(include='object').columns:
    dane[kolumna] = enkoder_etykiet.fit_transform(dane[kolumna])

dane = MinMaxScaler((-1,1)).fit_transform(dane)
y = dane[:, -1]
X = dane[:, :-1]

X_trenujace = X[0::3] + X[1::3]
y_trenujace = y[0::3] + y[1::3]
X_test = X[2::3]
y_test = y[2::3]

Przebieg testowy

In [ ]:
model = wielowarstwowy_perceptron(n_warstw=2, n_neuronow=3, n_wejsc=4, n_wyjsc=1, funkcja_aktywacji=sigmoid)
trenuj(model, X_trenujace, y_trenujace, 3, wspolczynnik_uczenia=0.01)
wynik = test(model, X_test, y_test)

##### Badanie celności działania sieci biorąc pod uwagę współczynnik uczenia
przy liczbie warstw równej 3, liczbie neuronów równej 8, funkcji aktywacji sigmoid i 1000 iteracji

In [ ]:
# x = []
# y = []
# for wspu in [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 1]:
#     model = wielowarstwowy_perceptron(n_warstw=3, n_neuronow=8, n_wejsc=4, n_wyjsc=1, funkcja_aktywacji=sigmoid)
#     trenuj(model, X_trenujace, y_trenujace, 1000, wspolczynnik_uczenia=wspu)
#     wynik = test(model, X_test, y_test)
#     x.append(wspu)
#     y.append(wynik)

# optymalny_wspolczynnik_uczenia = x[y.index(max(y))]
# print('Optymalny współczynnik uczenia: ', optymalny_wspolczynnik_uczenia)
# plt.ylabel('celność')
# plt.xlabel('współczynnik uczenia')
# plt.xticks(x, rotation='vertical')
# plt.plot(x, y)

##### Badanie celności działania sieci biorąc pod uwagę liczbę warstw
przy liczbie neuronów równej 8, funkcji aktywacji sigmoid, 1000 iteracji oraz optymalnym współczynniku uczenia

In [ ]:
# x = []
# y = []
# for n_warstw in [1, 2, 3, 4, 5, 8]:
#     model = wielowarstwowy_perceptron(n_warstw=n_warstw, n_neuronow=8, n_wejsc=4, n_wyjsc=1, funkcja_aktywacji=sigmoid)
#     trenuj(model, X_trenujace, y_trenujace, 1000, wspolczynnik_uczenia=optymalny_wspolczynnik_uczenia)
#     wynik = test(model, X_test, y_test)
#     x.append(n_warstw)
#     y.append(wynik)

# najlepsze_n_warstw = x[y.index(max(y))]
# plt.ylabel('celność')
# plt.xlabel('liczba warstw')
# plt.xticks(x, rotation='vertical')
# plt.plot(x, y)

##### Badanie celności działania sieci biorąc pod uwagę liczbę neuronów
przy najlepszej liczbie warstw, funkcji aktywacji sigmoid, 1000 iteracji oraz optymalnym współczynniku uczenia

In [ ]:
# x = []
# y = []
# for n_neuronow in [1, 2, 3, 4, 5, 8]:
#     model = wielowarstwowy_perceptron(n_warstw=najlepsze_n_warstw, n_neuronow=n_neuronow, n_wejsc=4, n_wyjsc=1, funkcja_aktywacji=sigmoid)
#     trenuj(model, X_trenujace, y_trenujace, 1000, wspolczynnik_uczenia=optymalny_wspolczynnik_uczenia)
#     wynik = test(model, X_test, y_test)
#     x.append(n_neuronow)
#     y.append(wynik)

# najlepsze_n_warstw = x[y.index(max(y))]
# plt.ylabel('celność')
# plt.xlabel('liczba neuronów')
# plt.xticks(x, rotation='vertical')
# plt.plot(x, y)